In [ ]:
# 출처 : https://wikidocs.net/233344
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [2]:
# 문서전체요약(Stuff Documents Chain)

# 요약관련 prompt를 https://smith.langchain.com/ 에서 내려 받는다.
'''
Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. Translate the summary into KOREAN if it is written in ENGLISH.
5. DO NOT translate any technical terms.
6. DO NOT include any unnecessary information.

CONTEXT:
{context}

SUMMARY:"
'''
from langchain import hub
prompt = hub.pull("teddynote/summary-stuff-documents-korean")


In [3]:
# TextLoader 를 사용하여 뉴스기사를 로드하고, Document를 생성합니다.
import os
from langchain_community.document_loaders import TextLoader

# ./data 폴더에 있는 .txt 확장자 파일을 모두 불러옴.
data_folder = "./data"
docs = []

for filename in os.listdir(data_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_folder, filename)
        loader = TextLoader(file_path)
        docs.extend(loader.load())

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

문서의 수: 2

[메타데이터]

{'source': './data/03.경조사지원규정_12.01.01.txt'}

========= [앞부분] 미리보기 =========

경조사 지원 규정
1. 적용대상
계약직 사원을 포함한 임직원(고문 및 용역은 사장이 별도로 결정)
2. 경조사 지원기준
구 분
내 역
휴가(일)
금액(원)
기타
경 사
본인 결혼
자녀 결혼
형제자매결혼
자녀 출산(배우자)
부모 회갑
배우자 부모회갑
부모고희(칠순)
배우자부모고희
5
1
1
10
1
1
1
1
500,000
300,000
100,000
100,000
200,000
200,000
300,000
300,000
화환 지급
화환 지급
1회 분할 사용 가능
조 사
본인 사망
배우자 사망
부모 사망
자녀 사망
배우자 부모 사망
형제자매 사망
조부모/외조부모 사망
-
5
5
5
5
3
3
1,000,000
1,000,000
1,000,000
1,000,000 500,000
300,000
200,000
조화 지급
조화 지급
조화 지급
조화 지급
조화 지급
조화 지급
3. 신청서류
경조금 : 경조금지급신청서 및 증빙서류 1부
휴 가: 휴가원 1부
4. 경조휴가일수
공휴일과 경조사가 중


In [4]:
# MyCallbackHandler 클래스는 BaseCallbackHandler를 상속받아, 언어 모델이 새로운 토큰을 생성할 때마다 해당 토큰을 출력하는 기능을 추가합니다.
# ChatOpenAI 객체는 GPT-3.5-turbo 모델을 사용하며, 스트리밍 모드와 낮은 온도 설정을 통해 더 일관된 응답을 생성하도록 구성됩니다. 
# 이 객체는 콜백으로 MyCallbackHandler 인스턴스를 사용합니다.
# 마지막으로, create_stuff_documents_chain 함수를 사용하여 문서 생성 체인을 만들고, 이 체인을 통해 주어진 문맥(docs)에 대한 응답(answer)을 생성합니다.

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.callbacks.base import BaseCallbackHandler

# 콜백 함수 설정
class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{token}", end="", flush=True)

# 모델 생성 
llm=ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,                  # 스트림=True로 설정
    temperature=0.01,
    callbacks=[MyCallbackHandler()], # 콜백함수 설정
)

# create_stuff_documents_chain 로 체인 생성
chain=create_stuff_documents_chain(llm, prompt)

# 예측
response=chain.invoke({"context": docs})
print(response)

📝 계약직 사원 및 임직원에게 경조금 및 휴가 제공
📝 경조금은 결혼, 출산, 사망 등에 대한 지원
📝 휴가는 공휴일 제외하여 계산
📝 결혼퇴직 시 결혼 후 1개월 이내에 지원
📝 특별한 경우 사업부장 합의 시 6개월 이내 신청 가능
📝 급여는 기본급, 제수당, 상여금, 퇴직금으로 구성
📝 급여는 매월 20일 지급, 휴일인 경우 전일로 지급
📝 퇴직금은 근속기간에 따라 계산하여 지급
📝 퇴직금은 평균임금 × 30 × 근속기간 / 365 계산하여 지급
📝 급여규정은 2007년 9월 14일부터 시행, 2016년 6월 1일부터 개정 및 시행"📝 계약직 사원 및 임직원에게 경조금 및 휴가 제공
📝 경조금은 결혼, 출산, 사망 등에 대한 지원
📝 휴가는 공휴일 제외하여 계산
📝 결혼퇴직 시 결혼 후 1개월 이내에 지원
📝 특별한 경우 사업부장 합의 시 6개월 이내 신청 가능
📝 급여는 기본급, 제수당, 상여금, 퇴직금으로 구성
📝 급여는 매월 20일 지급, 휴일인 경우 전일로 지급
📝 퇴직금은 근속기간에 따라 계산하여 지급
📝 퇴직금은 평균임금 × 30 × 근속기간 / 365 계산하여 지급
📝 급여규정은 2007년 9월 14일부터 시행, 2016년 6월 1일부터 개정 및 시행"
